In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
%%bash
samtools merge -f $TMPDIR/out.bam  \
BSF_06/res4/BSF_06/BSF_06_sorted.bam \
BSF_07/res4/BSF_07/BSF_07_sorted.bam \
BSF_08/res4/BSF_08/BSF_08_sorted.bam \

samtools sort -@ 10 -o $TMPDIR/BSF_G_merge.bam $TMPDIR/out.bam
rm $TMPDIR/out.bam
samtools index $TMPDIR/BSF_G_merge.bam

In [ ]:
#copy the library files to use as control
!cp BSF_01/res4/BSF_01/BSF_01_sorted.bam $TMPDIR/

In [ ]:
#create a folder to store macs result
!mkdir -p merge_macs_BSF_G

#use macs2 to call broad peaks
!macs2 callpeak -t $TMPDIR/BSF_G_merge.bam -f BAMPE \
-c $TMPDIR/BSF_01_sorted.bam \
--min-length 300 --max-gap 1 --broad --nomodel --keep-dup all\
-q 1 --broad-cutoff 1 --broad --llocal 3000 --outdir 'merge_macs_BSF_G'

In [ ]:
#load peak results and make a bed files
#set coverage to one for visualization
import pandas as pd
indf = pd.read_csv('merge_macs_BSF_G/NA_peaks.xls',sep='\t',comment='#')
#GeneID	Chr	Start	End	Strand
indf = indf[list(indf.columns[0:3])]
indf['coverage']=1
#indf.columns = ['GeneID','Chr','Start','End']
indf.to_csv('merge_macs_BSF_G/merge_macs_BSF_G.bed',sep='\t',index=False,header=False)
indf.head()

In [ ]:
#make SAF file to get read counts
indf = pd.read_csv('merge_macs_BSF_G/NA_peaks.xls',sep='\t',comment='#')
#GeneID	Chr	Start	End	Strand
indf = indf[[indf.columns[-1]] + list(indf.columns[0:3])]
indf.columns = ['GeneID','Chr','Start','End']
indf['Strand']='.'
indf.to_csv('merge_macs_BSF_G/merge_macs_BSF_G.SAF',sep='\t',index=False)
indf.head()

In [ ]:
!less merge_macs_BSF_G/merge_macs_BSF_G.SAF

In [ ]:
!featureCounts -h

In [ ]:
#we now count the read pairs for each B samples and barcoded B samples
!featureCounts -p -B -C -M -O -T 8 -F SAF -a 'merge_macs_BSF_G/merge_macs_BSF_G.SAF' \
-o 'merge_macs_BSF_G/merge_macs_BSF_G_counts.txt' \
/cluster/majf_lab/mtinti/UTR/BSF_01/res4/BSF_01/BSF_01_sorted.bam \
/cluster/majf_lab/mtinti/UTR/BSF_06/res4/BSF_06/BSF_06_sorted.bam \
/cluster/majf_lab/mtinti/UTR/BSF_07/res4/BSF_07/BSF_07_sorted.bam \
/cluster/majf_lab/mtinti/UTR/BSF_08/res4/BSF_08/BSF_08_sorted.bam \
/cluster/majf_lab/mtinti/UTR/BSF_01/res4/BSF_01/BSF_01_sorted_F.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_01/res4/BSF_01/BSF_01_sorted_R.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_01/res4/BSF_01/BSF_01_sorted_FR.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_01/res4/BSF_01/BSF_01_sorted_RR.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_06/res4/BSF_06/BSF_06_sorted_F.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_06/res4/BSF_06/BSF_06_sorted_R.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_06/res4/BSF_06/BSF_06_sorted_FR.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_06/res4/BSF_06/BSF_06_sorted_RR.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_07/res4/BSF_07/BSF_07_sorted_F.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_07/res4/BSF_07/BSF_07_sorted_R.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_07/res4/BSF_07/BSF_07_sorted_FR.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_07/res4/BSF_07/BSF_07_sorted_RR.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_08/res4/BSF_08/BSF_08_sorted_F.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_08/res4/BSF_08/BSF_08_sorted_R.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_08/res4/BSF_08/BSF_08_sorted_FR.bam  \
/cluster/majf_lab/mtinti/UTR/BSF_08/res4/BSF_08/BSF_08_sorted_RR.bam

In [ ]:
#get the counts of all samples
import pandas as pd
df = pd.read_csv('merge_macs_BSF_G/merge_macs_BSF_G_counts.txt',comment='#',sep='\t',index_col=[0])
counts_col = list(df.columns[5:])
new_names = [n.split('/')[-1].split('.')[0].replace('_sorted','') for n in counts_col]
new_names = dict(zip(counts_col,new_names))
df = df.rename(new_names,axis=1)
df.head()

,Chr,Start,End,Strand,Length,BSF_01,BSF_06,BSF_07,BSF_08,BSF_01_F,...,BSF_06_FR,BSF_06_RR,BSF_07_F,BSF_07_R,BSF_07_FR,BSF_07_RR,BSF_08_F,BSF_08_R,BSF_08_FR,BSF_08_RR
Geneid,,,,,,,,,,,,,,,,,,,,,
NA_peak_1,11L3_v3,248,660,.,413,207,455,363,310,0,...,17,1,0,3,10,1,0,0,16,2
NA_peak_2,11L3_v3,770,1914,.,1145,1117,4607,5774,3073,29,...,25,92,112,2,16,135,58,8,22,83
NA_peak_3,11L3_v3,27809,28704,.,896,22,122,74,72,0,...,1,5,2,0,0,1,6,0,1,5
NA_peak_4,11L3_v3,43681,44162,.,482,269,888,749,1165,0,...,45,20,0,0,35,19,3,0,71,25
NA_peak_5,11L3_v3,50638,50964,.,327,107,143,113,138,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Prepare a final table
final_table=pd.DataFrame(index=df.index)
final_table['Chr']=df.Chr
final_table['Start']=df.Start
final_table['End']=df.End
final_table.head()

,Chr,Start,End
Geneid,,,
NA_peak_1,11L3_v3,248,660
NA_peak_2,11L3_v3,770,1914
NA_peak_3,11L3_v3,27809,28704
NA_peak_4,11L3_v3,43681,44162
NA_peak_5,11L3_v3,50638,50964


In [ ]:
#prepare the control
final_table['Control_total']=df['BSF_01']
final_table['Control_F']=df['BSF_01_F']+df['BSF_01_RR']
final_table['Control_R']=df['BSF_01_R']+df['BSF_01_FR']
#prepare treatment
final_table['Treatment_total']=df['BSF_06']+df['BSF_07']+df['BSF_08']

final_table['Treatment_F']=(df['BSF_06_F']+df['BSF_07_F']+df['BSF_08_F']+
                            df['BSF_06_RR']+df['BSF_07_RR']+df['BSF_08_RR'])

final_table['Treatment_R']=(df['BSF_06_R']+df['BSF_07_R']+df['BSF_08_R']+
                            df['BSF_06_FR']+df['BSF_07_FR']+df['BSF_08_FR'])

#fraction of reads in forward oritentation
final_table['Treatment_fraction_F']=final_table['Treatment_F']/(final_table['Treatment_F']+final_table['Treatment_R'])
#by definition
final_table['Treatment_fraction_R']=1-final_table['Treatment_fraction_F']
final_table.head()

,Chr,Start,End,Control_total,Control_F,Control_R,Treatment_total,Treatment_F,Treatment_R,Treatment_fraction_F,Treatment_fraction_R
Geneid,,,,,,,,,,,
NA_peak_1,11L3_v3,248,660,207,3,7,1128,4,48,0.076923,0.923077
NA_peak_2,11L3_v3,770,1914,1117,58,20,13454,575,78,0.880551,0.119449
NA_peak_3,11L3_v3,27809,28704,22,0,1,268,25,2,0.925926,0.074074
NA_peak_4,11L3_v3,43681,44162,269,25,4,2802,67,151,0.307339,0.692661
NA_peak_5,11L3_v3,50638,50964,107,0,0,394,0,0,NaN,NaN


In [ ]:
final_table['Treatment_total_corrected_F']=final_table['Treatment_total']*final_table['Treatment_fraction_F']
final_table['Treatment_total_corrected_R']=final_table['Treatment_total']*final_table['Treatment_fraction_R']
final_table.head()

,Chr,Start,End,Control_total,Control_F,Control_R,Treatment_total,Treatment_F,Treatment_R,Treatment_fraction_F,Treatment_fraction_R,Treatment_total_corrected_F,Treatment_total_corrected_R
Geneid,,,,,,,,,,,,,
NA_peak_1,11L3_v3,248,660,207,3,7,1128,4,48,0.076923,0.923077,86.769231,1041.230769
NA_peak_2,11L3_v3,770,1914,1117,58,20,13454,575,78,0.880551,0.119449,11846.937213,1607.062787
NA_peak_3,11L3_v3,27809,28704,22,0,1,268,25,2,0.925926,0.074074,248.148148,19.851852
NA_peak_4,11L3_v3,43681,44162,269,25,4,2802,67,151,0.307339,0.692661,861.165138,1940.834862
NA_peak_5,11L3_v3,50638,50964,107,0,0,394,0,0,NaN,NaN,NaN,NaN


In [ ]:
#create a bed file
bed_file = df[['Chr', 'Start', 'End']]
bed_file=bed_file.reset_index()
bed_file = bed_file[['Chr', 'Start', 'End', 'Geneid']]
bed_file=bed_file.rename({'Geneid':'Peak_Id'},axis=1)
bed_file[['Chr', 'Start', 'End', 'Peak_Id']].to_csv('gla_peaks_final.bed', sep='\t',
                                                   header=False, index=False)
!head 'gla_peaks_final.bed'

11L3_v3	248	660	NA_peak_1
11L3_v3	770	1914	NA_peak_2
11L3_v3	27809	28704	NA_peak_3
11L3_v3	43681	44162	NA_peak_4
11L3_v3	50638	50964	NA_peak_5
11L3_v3	50975	51514	NA_peak_6
11L3_v3	52360	53489	NA_peak_7
11L3_v3	55134	56732	NA_peak_8
11L3_v3	58961	59515	NA_peak_9
11L3_v3	59964	60927	NA_peak_10


In [ ]:
bl =pd.read_csv('blacklisted gene.csv',engine='python',header=None)
bl.columns = ['gene','reason']
bl['gene']=[n.strip() for n in bl['gene']]
print(bl.shape)
bl.tail()

(284, 2)


,gene,reason
279,Tb927.9.15950,UTR_contained
280,Tb927.9.6570,UTR_contained
281,Tb927.9.6830,UTR_contained
282,Tb927.9.7500,UTR_contained
283,Tb927.9.7850,UTR_contained


In [ ]:
%%bash
bedtools intersect -a <(bedtools sort -i gla_peaks_final.bed) \
-b <(bedtools sort -i UTR3_final.bed) -wao > gla_peak_belongs_to_3UTR.bed

In [ ]:
#use awk to select only the fully contained
!awk '$(NF-2) != "-1"' gla_peak_belongs_to_3UTR.bed > gla_peak_belongs_to_3UTR_filtered.bed

In [ ]:
df = pd.read_csv('gla_peak_belongs_to_3UTR_filtered.bed',sep='\t',header=None)
df.columns = ['Chromosome','Peak_Start','Peak_End','Peak_Name',
              'UTR_Chromosome','UTR_Start','UTR_End','UTR_Gene_ID','UTR_Score','UTR_Strand','UTR_Peak_overlap']
del df['UTR_Score']
del df['UTR_Chromosome']
#df.head()

df_plus = df[df['UTR_Strand']=='+']
df_minus = df[df['UTR_Strand']=='-']
df_plus = df_plus.drop_duplicates(subset=['Peak_Name'],keep='first')
df_minus = df_minus.drop_duplicates(subset=['Peak_Name'],keep='last')
df = pd.concat([df_plus,df_minus])
df=df.sort_values(by=['Chromosome','Peak_Start','Peak_End'])
#df[df['Peak_Name']=='NA_peak_1371']

In [ ]:
print(df.shape)
df = df.merge(final_table.iloc[:,3:],left_on='Peak_Name',right_index=True,how='left')
print(df.shape)

(2212, 9)
(2212, 19)


In [ ]:
df.head(10)

,Chromosome,Peak_Start,Peak_End,Peak_Name,UTR_Start,UTR_End,UTR_Gene_ID,UTR_Strand,UTR_Peak_overlap,Control_total,Control_F,Control_R,Treatment_total,Treatment_F,Treatment_R,Treatment_fraction_F,Treatment_fraction_R,Treatment_total_corrected_F,Treatment_total_corrected_R
0,Tb927_01_v5.1,60141,61723,NA_peak_188,59853,60265,Tb927.1.120,-,124,4881,201,351,32261,1561,576,0.730463,0.269537,23565.475433,8695.524567
1,Tb927_01_v5.1,112710,113415,NA_peak_211,112744,112971,Tb927.1.280,+,227,2827,35,17,10330,164,35,0.824121,0.175879,8513.165829,1816.834171
2,Tb927_01_v5.1,144650,144961,NA_peak_228,144595,144725,Tb927.1.380,-,75,1985,45,23,6948,28,185,0.131455,0.868545,913.352113,6034.647887
3,Tb927_01_v5.1,149354,149874,NA_peak_231,148205,151108,Tb927.1.400,+,520,915,11,13,4968,40,35,0.533333,0.466667,2649.600000,2318.400000
4,Tb927_01_v5.1,150298,150742,NA_peak_232,148205,151108,Tb927.1.400,+,444,1241,70,34,5080,137,135,0.503676,0.496324,2558.676471,2521.323529
5,Tb927_01_v5.1,186068,188200,NA_peak_255,186030,187218,Tb927.1.480,+,1150,803,35,6,6706,39,54,0.419355,0.580645,2812.193548,3893.806452
6,Tb927_01_v5.1,190391,191332,NA_peak_257,190086,190499,Tb927.1.490,+,108,4160,14,109,20332,52,526,0.089965,0.910035,1829.176471,18502.823529
7,Tb927_01_v5.1,212303,213619,NA_peak_262,211572,212454,Tb927.1.580,-,151,763,79,62,9415,121,640,0.159001,0.840999,1496.997372,7918.002628
8,Tb927_01_v5.1,216015,217313,NA_peak_263,214185,216163,Tb927.1.600,-,148,707,8,40,7578,78,271,0.223496,0.776504,1693.650430,5884.349570
9,Tb927_01_v5.1,218980,219871,NA_peak_264,219177,219567,Tb927.1.630,-,390,110,1,8,802,13,50,0.206349,0.793651,165.492063,636.507937


In [ ]:
def format_download(X):
    if X['UTR_Strand']=='-':
        orient = 'r'
    else:
        orient = 'f'
    chro =  X['Chromosome']  
    p_start =  int(X['Peak_Start'])
    p_end =  int(X['Peak_End'])
    u_start =  int(X['UTR_Start'])
    u_end =  int(X['UTR_End'])  
    
    start = max(p_start,u_start)
    end = min(p_end,u_end)
    download_string = f'{chro}:{start}..{end}:{orient}'
    return download_string           

In [ ]:
df['download_seq']=df.apply(format_download,axis=1)

In [ ]:
def is_fully_contained(X):
    p_start =  int(X['Peak_Start'])
    p_end =  int(X['Peak_End'])
    u_start =  int(X['UTR_Start'])
    u_end =  int(X['UTR_End'])
    if (p_start>=u_start) and (p_end<=u_end):
        return True
    else:
        return False
df['is_fully_contained'] = df.apply(is_fully_contained,axis=1)

In [ ]:
df[df['is_fully_contained']].shape

(260, 21)

In [ ]:
print(df.shape)
df.head()

(2212, 21)


,Chromosome,Peak_Start,Peak_End,Peak_Name,UTR_Start,UTR_End,UTR_Gene_ID,UTR_Strand,UTR_Peak_overlap,Control_total,...,Control_R,Treatment_total,Treatment_F,Treatment_R,Treatment_fraction_F,Treatment_fraction_R,Treatment_total_corrected_F,Treatment_total_corrected_R,download_seq,is_fully_contained
0,Tb927_01_v5.1,60141,61723,NA_peak_188,59853,60265,Tb927.1.120,-,124,4881,...,351,32261,1561,576,0.730463,0.269537,23565.475433,8695.524567,Tb927_01_v5.1:60141..60265:r,False
1,Tb927_01_v5.1,112710,113415,NA_peak_211,112744,112971,Tb927.1.280,+,227,2827,...,17,10330,164,35,0.824121,0.175879,8513.165829,1816.834171,Tb927_01_v5.1:112744..112971:f,False
2,Tb927_01_v5.1,144650,144961,NA_peak_228,144595,144725,Tb927.1.380,-,75,1985,...,23,6948,28,185,0.131455,0.868545,913.352113,6034.647887,Tb927_01_v5.1:144650..144725:r,False
3,Tb927_01_v5.1,149354,149874,NA_peak_231,148205,151108,Tb927.1.400,+,520,915,...,13,4968,40,35,0.533333,0.466667,2649.600000,2318.400000,Tb927_01_v5.1:149354..149874:f,True
4,Tb927_01_v5.1,150298,150742,NA_peak_232,148205,151108,Tb927.1.400,+,444,1241,...,34,5080,137,135,0.503676,0.496324,2558.676471,2521.323529,Tb927_01_v5.1:150298..150742:f,True


In [ ]:
df.to_csv('Paper_Table_Ganciclovir.csv')

In [ ]:
# show versions of packages
# adopted from https://stackoverflow.com/questions/40428931/package-for-listing-version-of-packages-used-in-a-jupyter-notebook
import pkg_resources
import types
import sys
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]
        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]
        yield name.lower()
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
modules = []
for m in sys.builtin_module_names:
    if m.lower() in imports and m !='builtins':
        modules.append((m,'Python BuiltIn'))
        imports.remove(m.lower())

for m in pkg_resources.working_set:
    if m.project_name.lower() in imports and m.project_name!="pip":
        modules.append((m.project_name, m.version))
        imports.remove(m.project_name.lower())

for m in sys.modules:
    if m.lower() in imports and m !='builtins':
        modules.append((m,'unknown'))

# print('System=='+platform.system()+' '+platform.release()+'; Version=='+platform.version())
for r in modules:
    print("{}=={}".format(*r))

sys==Python BuiltIn
tqdm==4.64.0
matplotlib==3.5.1
numpy==1.21.6
pandas==1.4.2
types==unknown
pkg_resources==unknown


In [ ]:
#bedtools-2.31